In [1]:
import requests
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import matplotlib.ticker as ticker
from sklearn import preprocessing


In [2]:
!conda install -c anaconda beautifulsoup4 -y

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 anaconda

beautifulsoup4 100% |################################| Time: 0:00:00  20.12 MB/s


In [3]:
from bs4 import BeautifulSoup

In [4]:
post_codes_ca_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(post_codes_ca_url,'lxml')

In [5]:
codes_list=[]
borough_list=[]
neighborhood_list=[]
i=1
for tag in soup.table.find_all('td'):
    if i == 1:
        codes_list.append(tag.text)
    if i == 2:
        borough_list.append(tag.text)
    if i == 3: 
        neighborhood_list.append(tag.text)
    i = i+1
    if i==4:
        i=1

In [6]:
toronto_df = pd.DataFrame(columns=['Postalcode','Borough','Neighborhood'])
toronto_df['Postalcode'] = codes_list
toronto_df['Borough'] = borough_list
toronto_df['Neighborhood'] = neighborhood_list
#toronto_df

In [7]:
## Find the indexs of rows which Borough is 'Not assigned'
indexs = toronto_df[toronto_df['Borough']=="Not assigned"].index
## remove the rows with 'Not assigned' Borough
toronto_df.drop(toronto_df.index[indexs], inplace=True)

In [8]:
## remove the '\n' in Neighborhood at all rows
## reset index
toronto_df['Neighborhood'] = toronto_df.Neighborhood.str.replace("\n","")
toronto_df = toronto_df.reset_index(drop=True)
#toronto_df

In [9]:
## Get the indexs of which Neighborhood are 'Not assigned'
indexs_nb = toronto_df[toronto_df['Neighborhood']=="Not assigned"].index
#indexs_nb

In [10]:
## Define the Neighborhoods which are 'Not assgined' with value of Borough.
for i in range(212):
    if toronto_df.Neighborhood[i]=='Not assigned':
        toronto_df.Neighborhood[i] = toronto_df.Borough[i]

In [11]:
#toronto_df

In [12]:
## Combine the Neighborhood of same Posttal lines in one line. 
toronto_df_Final = toronto_df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(','.join).reset_index()
toronto_df_Final

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [13]:
## get all geographical coordinates of the neighborhoods.
geo_data_df = pd.read_csv('https://cocl.us/Geospatial_data')
#geo_data_df

In [14]:
## Merge the coordinates info in Postalcode/Borough/Neighborhood dataframe.
merged_df = pd.merge(toronto_df_Final, geo_data_df,left_on='Postalcode',right_on='Postal Code',how='left').drop('Postal Code', axis=1)
merged_df

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [15]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.86 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.04 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  53.72 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.85 MB/s
vincent-0.4.4- 100% |###################

In [16]:
## Define Foursquare Credentials and Version
CLIENT_ID = 'A0BEEBP0VG14B2EDHXDUX1YYNR0O5RLUV3CNMBUVX4UBOSGZ' # your Foursquare ID
CLIENT_SECRET = '3HELNYYP3L5ESUUC5RXE2ARB1HQQSQO4UOQQN2SOZB51T1DJ' # your Foursquare Secret
VERSION = '20180604'

In [17]:
address = 'Toronto, ON'

geolocator = Nominatim()
toronto = geolocator.geocode(address)
latitude = toronto.latitude
longitude = toronto.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#2156cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [19]:
# Plan to explore the Neighborhood 'Rouge,Malvern' where is at downtown Toronto, get its index first. 
merged_df[merged_df['Neighborhood']=='Rouge,Malvern'].index

Int64Index([0], dtype='int64')

In [20]:
neighborhood_latitude = merged_df.loc[75, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = merged_df.loc[75, 'Longitude'] # neighborhood longitude value

neighborhood_name = merged_df.loc[75, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Christie are 43.669542, -79.4225637.


In [21]:
LIMIT = 150
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=A0BEEBP0VG14B2EDHXDUX1YYNR0O5RLUV3CNMBUVX4UBOSGZ&client_secret=3HELNYYP3L5ESUUC5RXE2ARB1HQQSQO4UOQQN2SOZB51T1DJ&ll=43.653963,-79.387207&v=20180604&radius=500&limit=150'

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c5b99914434b9745dabdb23'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5227bb01498e17bf485e6202-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
          'suffix': '.png'},
         'id': '4f2a25ac4b909258e854f55f',
         'name': 'Neighborhood',
         'pluralName': 'Neighborhoods',
         'primary': True,
         'shortName': 'Neighborhood'}],
       'id': '5227bb01498e17bf485e6202',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 174,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng': -79.38529600606677}],
        'lat': 43.6532

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print(nearby_venues.head())
sorted_df=nearby_venues.sort_values('categories')
sorted_df['categories'].unique()

                       name        categories        lat        lng
0          Downtown Toronto      Neighborhood  43.653232 -79.385296
1  Textile Museum of Canada        Art Museum  43.654396 -79.386500
2        Sansotei Ramen 三草亭  Ramen Restaurant  43.655157 -79.386501
3                   Japango  Sushi Restaurant  43.655268 -79.385165
4               Cafe Plenty              Café  43.654571 -79.389450


array(['American Restaurant', 'Art Gallery', 'Art Museum',
       'Arts & Crafts Store', 'Asian Restaurant', 'Bakery', 'Bar',
       'Breakfast Spot', 'Bubble Tea Shop', 'Café', 'Chinese Restaurant',
       'Clothing Store', 'Coffee Shop', 'Concert Hall', 'Dessert Shop',
       'Donut Shop', 'Exhibit', 'Food Court', 'French Restaurant',
       'Gastropub', 'Greek Restaurant', 'Gym', 'Hotel',
       'Indian Restaurant', 'Japanese Restaurant', 'Jazz Club',
       'Korean Restaurant', 'Miscellaneous Shop',
       'Modern European Restaurant', 'Monument / Landmark',
       'Movie Theater', 'Neighborhood', 'Office', 'Opera House', 'Park',
       'Plaza', 'Poke Place', 'Pub', 'Ramen Restaurant', 'Record Shop',
       'Restaurant', 'Salon / Barbershop', 'Seafood Restaurant',
       'Smoke Shop', 'Steakhouse', 'Sushi Restaurant', 'Tapas Restaurant',
       'Tea Room', 'Thai Restaurant', 'Toy / Game Store', 'University',
       'Vegetarian / Vegan Restaurant'], dtype=object)

In [25]:
##Clean data: put similar categories together and delete none identifiable categories such as "Neighborhood, Miscellaneous Shop"

#sorted_df.replace('Café', 'Art2')
#sorted_df
sorted_df=nearby_venues.sort_values('categories')
sorted_df['categories'].unique()
sorted_df['cat2']=sorted_df['categories']
sorted_df.loc[sorted_df.cat2.str.contains('Art'), 'categories'] = 'Art'
sorted_df.loc[sorted_df.cat2.str.contains('Exhibit'), 'categories'] = 'Art'
sorted_df.loc[sorted_df.cat2.str.contains('Café'), 'categories'] = 'Coffee Shop'
sorted_df.loc[sorted_df.cat2.str.contains('Sushi Restaurant'), 'categories'] = 'Japaneses Restaurant'

sort2=sorted_df[sorted_df.cat2 != 'Neighborhood']

nearby_venues=sort2.drop(columns=['cat2'])
nearby_venues


,name,categories,lat,lng
47,JOEY,American Restaurant,43.656094,-79.381878
18,Nota Bene,American Restaurant,43.650512,-79.388131
54,AGO - David Bowie Is Exhibition,Art,43.654018,-79.392850
45,Norma Ridley Members’ Lounge,Art,43.652786,-79.392293
62,Art Square Gallery & Cafe,Art,43.654227,-79.392536
19,Art Gallery of Ontario,Art,43.654003,-79.392922
1,Textile Museum of Canada,Art,43.654396,-79.386500
11,Aboveground Art Supplies,Art,43.652646,-79.390925
37,Noodle King,Asian Restaurant,43.651706,-79.383046
31,Butter Baker,Bakery,43.655669,-79.384298


In [34]:
## create a function to repeat the same process to all the neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'categories']
    
    return(nearby_venues)

In [35]:
# find the venues in all Toronto Neighbohoods
toronto_venues = getNearbyVenues(names=merged_df['Neighborhood'],
                                   latitudes=merged_df['Latitude'],
                                   longitudes=merged_df['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West,Steeles West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beac

In [37]:
toronto_venues.head()
#sorted_df=toronto_venues.sort_values('Venue Category')

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,categories
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [38]:
sorted_df=toronto_venues.sort_values('categories')
sorted_df['categories'].unique()


array(['Accessories Store', 'Adult Boutique', 'Afghan Restaurant',
       'Airport', 'Airport Food Court', 'Airport Gate', 'Airport Lounge',
       'Airport Service', 'Airport Terminal', 'American Restaurant',
       'Antique Shop', 'Aquarium', 'Arepa Restaurant', 'Art Gallery',
       'Art Museum', 'Arts & Crafts Store', 'Asian Restaurant',
       'Athletics & Sports', 'Auto Garage', 'Auto Workshop', 'BBQ Joint',
       'Baby Store', 'Bagel Shop', 'Bakery', 'Bank', 'Bar',
       'Baseball Field', 'Baseball Stadium', 'Basketball Court',
       'Basketball Stadium', 'Beach', 'Beer Bar', 'Beer Store',
       'Belgian Restaurant', 'Bike Shop', 'Bistro', 'Board Shop',
       'Boat or Ferry', 'Bookstore', 'Boutique', 'Brazilian Restaurant',
       'Breakfast Spot', 'Brewery', 'Bridal Shop', 'Bubble Tea Shop',
       'Building', 'Burger Joint', 'Burrito Place', 'Bus Line',
       'Bus Station', 'Bus Stop', 'Butcher', 'Cafeteria', 'Café',
       'Cajun / Creole Restaurant', 'Candy Store', 'Ca

In [40]:
sorted_df['cat2']=sorted_df['categories']
sorted_df.loc[sorted_df.cat2.str.contains('Art'), 'categories'] = 'Art'
sorted_df.loc[sorted_df.cat2.str.contains('Exhibit'), 'categories'] = 'Art'
sorted_df.loc[sorted_df.cat2.str.contains('Music'), 'categories'] = 'Music'
sorted_df.loc[sorted_df.cat2.str.contains('Bar'), 'categories'] = 'Bar'
sorted_df.loc[sorted_df.cat2.str.contains('Pub'), 'categories'] = 'Bar'
sorted_df.loc[sorted_df.cat2.str.contains('Café'), 'categories'] = 'Coffee Shop'
sorted_df.loc[sorted_df.cat2.str.contains('Sushi Restaurant'), 'categories'] = 'Japaneses Restaurant'
sorted_df.loc[sorted_df.cat2.str.contains('Dim Sum Restaurant'), 'categories'] = 'Chinese Restaurant'
sorted_df.loc[sorted_df.cat2.str.contains('Dumpling Restaurant'), 'categories'] = 'Chinese Restaurant'
sorted_df.loc[sorted_df.cat2.str.contains('Taiwanese Restaurant'), 'categories'] = 'Chinese Restaurant'
sort2=sorted_df[sorted_df.cat2 != 'Neighborhood']

toronto_venues=sort2.drop(columns=['cat2'])
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,categories
56,"Maryvale,Wexford",43.750072,-79.295849,Puffin Gear,43.750947,-79.290047,Accessories Store
145,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,Laura,43.777834,-79.343973,Accessories Store
1865,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,Ardene Shoes Outlet,43.718892,-79.461344,Accessories Store
614,Church and Wellesley,43.665860,-79.383160,Seduction,43.665620,-79.384681,Adult Boutique
644,Church and Wellesley,43.665860,-79.383160,Naan & Kabob,43.669005,-79.386219,Afghan Restaurant


In [41]:
## check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,categories
Neighborhood,,,,,,
"Adelaide,King,Richmond",99,99,99,99,99,99
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",12,12,12,12,12,12
"Alderwood,Long Branch",10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55


In [42]:
## find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['categories'].unique())))

There are 253 uniques categories.


In [43]:
## Analyze Each Neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['categories']], prefix="", prefix_sep="")
print(toronto_onehot.columns)
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
column_list = toronto_onehot.columns.tolist()
column_number = int(column_list.index('Neighborhood'))
column_list = [column_list[column_number]] + column_list[:column_number] + column_list[column_number+1:]
#sorted_df = toronto_onehot.sort_values(['Chinese Restaurant','Neighborhood','Wine Bar', 'Yoga Studio'], ascending=[0,1,0,0])

#sorted_df[['Chinese Restaurant','Neighborhood']]

Index(['Accessories Store', 'Adult Boutique', 'Afghan Restaurant', 'Airport',
       'Airport Food Court', 'Airport Gate', 'Airport Lounge',
       'Airport Service', 'Airport Terminal', 'American Restaurant',
       ...
       'Trail', 'Train Station', 'Vegetarian / Vegan Restaurant',
       'Video Game Store', 'Video Store', 'Vietnamese Restaurant',
       'Warehouse Store', 'Wings Joint', 'Women's Store', 'Yoga Studio'],
      dtype='object', length=253)


In [44]:
## group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#sorted_df = toronto_grouped.sort_values(['Chinese Restaurant','Neighborhood','Wine Bar', 'Yoga Studio'], ascending=[0,1,0,0])

#sorted_df[['Chinese Restaurant','Neighborhood']]

In [45]:
## print each neighborhood along with the top 5 most common venues

num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.11
1                  Bar  0.06
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04
5     Asian Restaurant  0.03
6                Hotel  0.03
7               Bakery  0.03
8       Clothing Store  0.03
9                  Gym  0.03


----Agincourt----
                             venue  freq
0               Chinese Restaurant  0.25
1                   Sandwich Place  0.25
2                           Lounge  0.25
3                   Breakfast Spot  0.25
4                Accessories Store  0.00
5              Monument / Landmark  0.00
6  Molecular Gastronomy Restaurant  0.00
7       Modern European Restaurant  0.00
8                Mobile Phone Shop  0.00
9        Middle Eastern Restaurant  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                             venue  freq
0                             Park  0.33
1                      Coffee Sho

In [46]:
## put that into a pandas dataframe
## First, write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
## create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Bar,Thai Restaurant,Steakhouse,American Restaurant,Gym,Asian Restaurant,Bakery,Clothing Store,Restaurant
1,Agincourt,Chinese Restaurant,Lounge,Sandwich Place,Breakfast Spot,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Coffee Shop,Playground,Doner Restaurant,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Drugstore
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fried Chicken Joint,Coffee Shop,Pharmacy,Pizza Place,Discount Store,Sandwich Place,Beer Store,Japanese Restaurant,Fast Food Restaurant
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Bar,Gym,Skating Rink,Athletics & Sports,Pharmacy,Pool,Sandwich Place,Yoga Studio
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Grocery Store,Frozen Yogurt Shop,Bridal Shop,Fast Food Restaurant,Sandwich Place,Japaneses Restaurant,Restaurant,Diner,Bank
6,Bayview Village,Chinese Restaurant,Japanese Restaurant,Coffee Shop,Bank,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
7,"Bedford Park,Lawrence Manor East",Coffee Shop,Italian Restaurant,Bar,Japaneses Restaurant,Fast Food Restaurant,Grocery Store,Hardware Store,Indian Restaurant,Liquor Store,Comfort Food Restaurant
8,Berczy Park,Bar,Coffee Shop,Restaurant,Steakhouse,Cheese Shop,Italian Restaurant,Seafood Restaurant,Bakery,Farmers Market,Jazz Club
9,"Birch Cliff,Cliffside West",College Stadium,Coffee Shop,Skating Rink,General Entertainment,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Eastern European Restaurant


In [50]:
## Cluster Neighborhoods
## Run k-means to cluster the neighborhood into 10 clusters.

# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 8, 1, 1, 1, 8, 1, 1, 8], dtype=int32)

In [51]:
# add clustering labels
toronto_merged = merged_df.head(100)
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.sort_values(['Cluster Labels', '1st Most Common Venue','2nd Most Common Venue'])

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
80,M6M,York,"Del Ray,Keelesdale,Mount Dennis,Silverthorn",43.691116,-79.476013,0,Coffee Shop,Restaurant,Check Cashing Service,Discount Store,Sandwich Place,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
68,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420,1,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Plane,Boutique,Airport Gate,Airport
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Bakery,Bank,Thai Restaurant,Caribbean Restaurant,Donut Shop,Dog Run,Doner Restaurant
94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724,1,Bank,Yoga Studio,Field,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,1,Bar,Yoga Studio,Field,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
97,M9M,North York,"Emery,Humberlea",43.724766,-79.532242,1,Baseball Field,Yoga Studio,Field,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,1,Bus Line,Bakery,Soccer Field,Bus Station,Intersection,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Empanada Restaurant,Donut Shop
20,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714,1,Cafeteria,Yoga Studio,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
15,M1W,Scarborough,"L'Amoreaux West,Steeles West",43.799525,-79.318389,1,Chinese Restaurant,Grocery Store,Fast Food Restaurant,Nail Salon,Japanese Restaurant,Sandwich Place,Breakfast Spot,Thrift / Vintage Store,Electronics Store,Pizza Place
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,1,Chinese Restaurant,Lounge,Sandwich Place,Breakfast Spot,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


In [52]:
# create final map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-2],
        fill=True,
        fill_color=rainbow[cluster-2],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Conclusion
## Downtown offers everything--All kind of categories can be found in downtown Toronto 
## Toronto is a super convenient city with almost all the neighborhood equiped with all kind of business near by--Category 1 is the most diversified category
